# Machine learning Modeling and Testing
The goal of tthis notebook is to create a group of models who can predict if a text was writen by a person with depression. The input dataset was found in Kaggle (https://www.kaggle.com/datasets/infamouscoder/depression-reddit-cleaned).

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!python -m spacy download en_core_web_lg

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2022-12-27 20:32:27.936137: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 587.7 MB 15 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [ ]:
import pandas as pd  
import numpy as np

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [20, 7]

import re

import nltk
nltk.download('punkt')
nltk.download('stopwords')

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import plot_precision_recall_curve
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score, classification_report

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras import optimizers
import tensorflow.keras
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

import spacy

from scipy import sparse
from scipy import stats

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [ ]:
# Loads the csv into a pandas dataframe.
#df = pd.read_csv('../Data/depression_dataset_reddit_cleaned.csv')
df = pd.read_csv('/content/drive/MyDrive/Enhance IT Data Science Course/Preparation (1)/BetterHelp/Data/depression_dataset_reddit_cleaned.csv')

In [ ]:
df.head()

,clean_text,is_depression
0,we understand that most people who reply immed...,1
1,welcome to r depression s check in post a plac...,1
2,anyone else instead of sleeping more when depr...,1
3,i ve kind of stuffed around a lot in my life d...,1
4,sleep is my greatest and most comforting escap...,1


In [ ]:
cleaned = []
lemwn = WordNetLemmatizer()
nlp = spacy.load('en_core_web_lg')
for i in df['clean_text']:
    text = []
    # Tokenizes the text
    text = word_tokenize(i)
    
    # Eliminates punctuations, roman numbers, and lowcaps every letter.
    nptext = []
    for token in text:
        new_token = re.sub('[^\w\s]', '', token)
        new_token = re.sub('^(?=[MDCLXVI])M*(C[MD]|D?C*)(X[CL]|L?X*)(I[XV]|V?I*)$', '', new_token)
        new_token = new_token.lower()

        new_token = re.sub('(?:^|\W)wa(?:$|\W)', 'was', new_token)

        if not new_token == '':
            nptext.append(new_token)
    text = nptext
    
    # Eliminates arabic numbers
    text = [x for x in text if not x.isnumeric()]
    
    # Eliminates stop words
    stops = set(stopwords.words('english')) #are all lower case
    text = [token for token in text if token not in stops]
    
    # De-tokenize the list and puts everything into a string.
    string = ' '.join(text)

    # lemmatization
    text = []
    for i, token in enumerate(nlp(string)):
        text.append(token.lemma_)

    # De-tokenize the list and puts everything into a string.
    string = ' '.join(text)
    
    # Cleaned string appendization to the list of cleaned tweets
    cleaned.append(string)

In [ ]:
print(cleaned[:20])

['understand people reply immediately op invitation talk privately mean help type response usually lead either disappointment disaster usually work quite differently say pm anytime casual social context huge admiration appreciation goodwill good citizenship many support other flag inappropriate content even know many struggle hard work behind scene information resource make easy give get quality help small start new wiki page explain detail much well respond public comment least get know someone maintain r depression wiki private contact full text current version summary anyone act helper invite accept private contact e pm chat kind offsite communication early conversion show either bad intention bad judgement either way unwise trust pm anytime seem like kind generous offer might perfectly well mean unless solid rapport ha establish wise idea point consider offer accept invitation communicate privately post supportive reply publicly help people op response good quality educate inspire 

In [ ]:
# Creation of a new columnn named 'text'. This column will be inserted by each one of the items with the clean text of their respective
df.insert(loc = 2, value = cleaned, column = 'text')
df

,clean_text,is_depression,text
0,we understand that most people who reply immed...,1,understand people reply immediately op invitat...
1,welcome to r depression s check in post a plac...,1,welcome r depression check post place take mom...
2,anyone else instead of sleeping more when depr...,1,anyone else instead sleep depressed stay night...
3,i ve kind of stuffed around a lot in my life d...,1,kind stuff around lot life delay inevitable wo...
4,sleep is my greatest and most comforting escap...,1,sleep great comforting escape whenever wake da...
...,...,...,...
7726,is that snow,0,snow
7727,moulin rouge mad me cry once again,0,moulin rouge mad cry
7728,trying to shout but can t find people on the list,0,try shout find people list
7729,ughh can t find my red sox hat got ta wear thi...,0,ughh find red sox hat get to wear creepy nick ...


In [ ]:
# Creation of a new clomun named 'text_length' which displays the length of each text in the column 'text'.
df['text_length'] = [len(i) for i in df['text']]
df

,clean_text,is_depression,text,text_length
0,we understand that most people who reply immed...,1,understand people reply immediately op invitat...,2971
1,welcome to r depression s check in post a plac...,1,welcome r depression check post place take mom...,1513
2,anyone else instead of sleeping more when depr...,1,anyone else instead sleep depressed stay night...,143
3,i ve kind of stuffed around a lot in my life d...,1,kind stuff around lot life delay inevitable wo...,277
4,sleep is my greatest and most comforting escap...,1,sleep great comforting escape whenever wake da...,152
...,...,...,...,...
7726,is that snow,0,snow,4
7727,moulin rouge mad me cry once again,0,moulin rouge mad cry,20
7728,trying to shout but can t find people on the list,0,try shout find people list,26
7729,ughh can t find my red sox hat got ta wear thi...,0,ughh find red sox hat get to wear creepy nick ...,59


In [ ]:
test_df = df[:5]
test_df = pd.concat([test_df, df[7726:]])
test_df

,clean_text,is_depression,text,text_length
0,we understand that most people who reply immed...,1,understand people reply immediately op invitat...,2971
1,welcome to r depression s check in post a plac...,1,welcome r depression check post place take mom...,1513
2,anyone else instead of sleeping more when depr...,1,anyone else instead sleep depressed stay night...,143
3,i ve kind of stuffed around a lot in my life d...,1,kind stuff around lot life delay inevitable wo...,277
4,sleep is my greatest and most comforting escap...,1,sleep great comforting escape whenever wake da...,152
7726,is that snow,0,snow,4
7727,moulin rouge mad me cry once again,0,moulin rouge mad cry,20
7728,trying to shout but can t find people on the list,0,try shout find people list,26
7729,ughh can t find my red sox hat got ta wear thi...,0,ughh find red sox hat get to wear creepy nick ...,59
7730,slept wonderfully finally tried swatching for ...,0,sleep wonderfully finally try swatche new proj...,95


In [ ]:
df = df[5:7726]

In [ ]:
df

,clean_text,is_depression,text,text_length
5,i m year old turning soon in a few month i liv...,1,year old turn soon month live constant dread p...,956
6,i live alone and despite me being prone to lon...,1,live alone despite prone loneliness find emoti...,321
7,i m not looking for sympathy just simply to st...,1,look sympathy simply state do try survive sit ...,336
8,i don t know how to communicate all of my thou...,1,know communicate think stay inside instead tel...,277
9,mom i m sad it hurt in my heart the feeling fa...,1,mom sad hurt heart feeling fall stomach stop t...,433
...,...,...,...,...
7721,nz the place to be miss my own bed a bit though,0,nz place miss bed bit though,28
7722,just read r s amazing blog so tired don t want...,0,read r amazing blog tired want go school tomor...,62
7723,i hate being awake going to school western civ...,0,hate awake go school western civ presentation ...,71
7724,ha just discovered the downside of going away ...,0,ha discover downside go away weekend food shop...,64


In [ ]:
len(df)

7721

In [ ]:
x = df.text
y = df.is_depression

In [ ]:
SEED = 999
x_train, x_validation, y_train, y_validation = train_test_split(x, y, test_size=.2, random_state=SEED)

In [ ]:
tvec1 = TfidfVectorizer(max_features=2000, ngram_range=(1, 3))
tvec1.fit(x_train)

TfidfVectorizer(max_features=2000, ngram_range=(1, 3))

In [ ]:
x_train_tfidf = tvec1.transform(x_train)

In [ ]:
x_train_tfidf

<6176x2000 sparse matrix of type '<class 'numpy.float64'>'
	with 148658 stored elements in Compressed Sparse Row format>

In [ ]:
tvec2 = TfidfVectorizer(max_features=2000,ngram_range=(1, 3))
tvec2.fit(x_validation)

TfidfVectorizer(max_features=2000, ngram_range=(1, 3))

In [ ]:
x_validation_tfidf = tvec1.transform(x_validation).toarray()

In [ ]:
%%time
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()
clf.fit(x_train_tfidf, y_train)

CPU times: user 76.1 ms, sys: 23 µs, total: 76.1 ms
Wall time: 82.3 ms


LogisticRegression()

In [ ]:
clf.score(x_validation_tfidf, y_validation)

0.9540453074433657

In [ ]:
pred_log = clf.predict(x_validation_tfidf)
precision = precision_score(y_validation, pred_log)
recall = recall_score(y_validation, pred_log)
print('Precision: ',precision)
print('Recall: ',recall)

Precision:  0.9806629834254144
Recall:  0.9256844850065189


In [ ]:
print(classification_report(y_validation, clf.predict(x_validation_tfidf)))

              precision    recall  f1-score   support

           0       0.93      0.98      0.96       778
           1       0.98      0.93      0.95       767

    accuracy                           0.95      1545
   macro avg       0.96      0.95      0.95      1545
weighted avg       0.96      0.95      0.95      1545



In [ ]:
def batch_generator(X_data, y_data, batch_size):
    samples_per_epoch = X_data.shape[0]
    number_of_batches = samples_per_epoch/batch_size
    counter=0
    index = np.arange(np.shape(y_data)[0])
    while 1:
        index_batch = index[batch_size*counter:batch_size*(counter+1)]
        X_batch = X_data[index_batch,:].toarray()
        y_batch = y_data[y_data.index[index_batch]]
        counter += 1
        yield np.array(X_batch), np.array(y_batch)
        if (counter > number_of_batches):
            counter=0

In [ ]:
%%time
model = Sequential()
model.add(Dense(64, activation='relu', input_dim=2000))
model.add(Dropout(0.9))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.9))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

a =model.fit_generator(generator=batch_generator(x_train_tfidf, y_train, 32),
                    epochs=25, validation_data=(x_validation_tfidf, y_validation),
                    steps_per_epoch=x_train_tfidf.shape[0]/32)

Epoch 1/25
193/193 [==============================] - 2s 8ms/step - loss: 0.6887 - accuracy: 0.5308 - val_loss: 0.6627 - val_accuracy: 0.8854
Epoch 2/25
193/193 [==============================] - 1s 7ms/step - loss: 0.6313 - accuracy: 0.6274 - val_loss: 0.5234 - val_accuracy: 0.9236
Epoch 3/25
193/193 [==============================] - 1s 7ms/step - loss: 0.5171 - accuracy: 0.7440 - val_loss: 0.3237 - val_accuracy: 0.9417
Epoch 4/25
193/193 [==============================] - 1s 6ms/step - loss: 0.4120 - accuracy: 0.8146 - val_loss: 0.2045 - val_accuracy: 0.9540
Epoch 5/25
193/193 [==============================] - 2s 9ms/step - loss: 0.3564 - accuracy: 0.8467 - val_loss: 0.1610 - val_accuracy: 0.9586
Epoch 6/25
193/193 [==============================] - 2s 10ms/step - loss: 0.3078 - accuracy: 0.8745 - val_loss: 0.1352 - val_accuracy: 0.9573
Epoch 7/25
193/193 [==============================] - 1s 7ms/step - loss: 0.2752 - accuracy: 0.8861 - val_loss: 0.1209 - val_accuracy: 0.9592
Epoch

In [ ]:
#Accuracy score
original_score = max(a.history['val_accuracy'])
original_score

0.9637540578842163

In [ ]:
pred_nn = model.predict(x_validation_tfidf)
precision = precision_score(y_validation, np.round_(pred_nn))
recall = recall_score(y_validation, np.round_(pred_nn))
print('Precision: ',precision)
print('Recall: ',recall)

49/49 [==============================] - 0s 2ms/step
Precision:  0.9720372836218375
Recall:  0.9517601043024772


In [ ]:
def cleantext(phrase):
    lemwn = WordNetLemmatizer()
    nlp = spacy.load('en_core_web_lg')
    text = []
    # Tokenizes the text
    text = word_tokenize(phrase)
    
    # Eliminates punctuations, roman numbers, and lowcaps every letter.
    nptext = []
    for token in text:
        new_token = re.sub('[^\w\s]', '', token)
        new_token = re.sub('^(?=[MDCLXVI])M*(C[MD]|D?C*)(X[CL]|L?X*)(I[XV]|V?I*)$', '', new_token)
        new_token = new_token.lower()

        new_token = re.sub('(?:^|\W)wa(?:$|\W)', 'was', new_token)

        if not new_token == '':
            nptext.append(new_token)
    text = nptext
    
    # Eliminates arabic numbers
    text = [x for x in text if not x.isnumeric()]
    
    # Eliminates stop words
    stops = set(stopwords.words('english')) #are all lower case
    text = [token for token in text if token not in stops]
    
    # De-tokenize the list and puts everything into a string.
    string = ' '.join(text)

    # lemmatization
    text = []
    for i, token in enumerate(nlp(string)):
        text.append(token.lemma_)

    # De-tokenize the list and puts everything into a string.
    string = ' '.join(text)
    
    # Returns the clean value
    return(string)

In [ ]:
def logregprediction(text):
    # Cleans the text
    clean_text = cleantext(text)
    # Vectorizes the cleaned text
    vector = tvec1.transform([clean_text])
    
    # Returns the prediction of the logistic regression model
    answer = clf.predict(vector)[0]
    return 'Depression' if answer == 1 else 'No depression'

In [ ]:
def nnprediction(text):
    # Cleans the text
    clean_text = cleantext(text)
    # Vectorizes the cleaned text
    vector = tvec1.transform([clean_text])
    sparse.csr_matrix.sort_indices(vector)
    # Returns the prediction of the logistic regression model 
    answer = model.predict(vector, verbose = 0)[0][0]
    return 'Depression' if round(answer) == 1 else 'No depression'

In [ ]:
text = input('Please input your text: ')
print('Logistic regression: ' + str(logregprediction(text)))
print('Neural network: '  + str(nnprediction(text)))

Please input your text: There is a super long line and not enough time. I'm sorry. I'm not gonna be able to go back and talk to him again.
Logistic regression: No depression
Neural network: No depression


In [ ]:
test_df

,clean_text,is_depression,text,text_length
0,we understand that most people who reply immed...,1,understand people reply immediately op invitat...,2971
1,welcome to r depression s check in post a plac...,1,welcome r depression check post place take mom...,1513
2,anyone else instead of sleeping more when depr...,1,anyone else instead sleep depressed stay night...,143
3,i ve kind of stuffed around a lot in my life d...,1,kind stuff around lot life delay inevitable wo...,277
4,sleep is my greatest and most comforting escap...,1,sleep great comforting escape whenever wake da...,152
7726,is that snow,0,snow,4
7727,moulin rouge mad me cry once again,0,moulin rouge mad cry,20
7728,trying to shout but can t find people on the list,0,try shout find people list,26
7729,ughh can t find my red sox hat got ta wear thi...,0,ughh find red sox hat get to wear creepy nick ...,59
7730,slept wonderfully finally tried swatching for ...,0,sleep wonderfully finally try swatche new proj...,95


In [ ]:
# Change the '1, 0' values from the actual values in the 'is_depression' column.
test_df['is_depression'] = test_df['is_depression'].replace([1, 0], ['Depression', 'No depression'])

# Drops the 'text' and 'text_length' columns.
test_df = test_df.drop(['text', 'text_length'], axis = 1)

In [ ]:
lr_results = []
nn_results = []

for text in test_df['clean_text']:
    lr_results.append(logregprediction(text))
    nn_results.append(nnprediction(text))

test_df['lr_result'] = lr_results
test_df['nn_result'] = nn_results

In [ ]:
test_df

,clean_text,is_depression,lr_result,nn_result
0,we understand that most people who reply immed...,Depression,Depression,Depression
1,welcome to r depression s check in post a plac...,Depression,Depression,Depression
2,anyone else instead of sleeping more when depr...,Depression,Depression,Depression
3,i ve kind of stuffed around a lot in my life d...,Depression,Depression,Depression
4,sleep is my greatest and most comforting escap...,Depression,Depression,Depression
7726,is that snow,No depression,No depression,No depression
7727,moulin rouge mad me cry once again,No depression,No depression,No depression
7728,trying to shout but can t find people on the list,No depression,No depression,No depression
7729,ughh can t find my red sox hat got ta wear thi...,No depression,No depression,No depression
7730,slept wonderfully finally tried swatching for ...,No depression,No depression,No depression
